In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
from datetime import timedelta
from datetime import datetime

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
person_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/person-headers.txt', delimiter='\t')

In [ ]:
person_columns = person_headers.columns

In [ ]:
event_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/event-headers.txt', delimiter='\t')
event_columns = event_headers.columns

In [ ]:
touch_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/touch_headers.txt', delimiter='\t')
touch_columns = touch_headers.columns

In [ ]:
person_ranking_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/person_ranking.tsv',
                                     delimiter='\t')
person_ranking_columns = person_ranking_headers.columns

In [ ]:
person_ranking_item_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/person_ranking_item.tsv', 
                                          delimiter='\t')
person_ranking_item_columns = person_ranking_item_headers.columns

In [ ]:
person_ranking_summary_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/person_ranking_summary.tsv', 
                                          delimiter='\t')
person_ranking_summary_columns = person_ranking_summary_headers.columns

In [ ]:
person_ranking_columns

In [ ]:
person_ranking_summary_columns

In [ ]:
person_ranking_summary_headers.head()

Looks to me like the following tables have been cleaned/analyzed:
* person_identifier
* touch
* matched_segment

currently working on:
* person

In [ ]:

person_ids_gt_one_sorted = dd.read_csv('Z:\\person_ids_gt_one_sorted.csv',names=['person_id'], dtype={'person_id':np.float64})


In [ ]:
person_ranking_summ_headers = "person_id, customer_id, customer_site_id, site_name, site_external_id, column_meta_data_id, column_meta_data_name, item_name, frequency, freq_rank, last_modified_date, db_last_modified_date".split(', ')

In [ ]:
person_ranking_summ = dd.read_csv('Z:\\person_ranking_summary.tsv', names=person_ranking_summ_headers, delimiter='\t',
                                  dtype={'column_meta_data_id':np.float64, 'freq_rank':np.float64, 'frequency':np.float64, 'person_id':np.float64})

In [ ]:
person_ranking_summ[person_ranking_summ['person_id'].isin(person_ids_gt_one_sorted['person_id'].compute())].to_csv('Z:\\person_ranking_summary_filtered.tsv')

In [ ]:
person_ranking_summ_filtered =  dd.read_csv('Z:\\prs_filtered.tsv', header=0, names= ['']+person_ranking_summ_headers, delimiter=',',
                                  dtype={'':np.float64, 'column_meta_data_id':np.float64, 'freq_rank':np.float64, 'frequency':np.float64, 'person_id':np.float64})
#prs_value_counts = person_ranking_summ_filtered.item_name.value_counts().compute()
#for x in prs_value_counts.index:
#    print(x, prs_value_counts[x])

In [ ]:
person_ranking_summ_filtered = person_ranking_summ_filtered.drop_duplicates(['person_id', 'item_name'])


In [ ]:

#drop duplicates of the person_id, item_id
prs_value_counts = person_ranking_summ_filtered[person_ranking_summ_filtered['column_meta_data_id'].isin([37337, 37341, 37336, 37324, 38561, 39943, 39645])].item_name.value_counts().compute()
for x in prs_value_counts.index:
    print(x, prs_value_counts[x])

## looking at the distribution of terms across people who search in en vs es

In [ ]:
people_en = person_ranking_summ_filtered[person_ranking_summ_filtered['item_name'] == 'en']['person_id'].value_counts().index.compute()
prsf_en = person_ranking_summ_filtered[person_ranking_summ_filtered['person_id'].isin(people_en)]

people_es = person_ranking_summ_filtered[person_ranking_summ_filtered['item_name'] == 'es']['person_id'].value_counts().index.compute()
prsf_es = person_ranking_summ_filtered[person_ranking_summ_filtered['person_id'].isin(people_es)]


In [ ]:
prs_value_counts_en = prsf_en[prsf_en['column_meta_data_id'].isin([37337])].item_name.value_counts().compute()
for x in prs_value_counts_en.index:
    print(x, prs_value_counts_en[x])

In [ ]:
prs_value_counts_es = prsf_es[prsf_es['column_meta_data_id'].isin([37337])].item_name.value_counts().compute()
for x in prs_value_counts_es.index:
    print(x, prs_value_counts_es[x])

In [ ]:
%matplotlib inline


prs_value_counts_en[prs_value_counts_en.index[:100]].plot(kind='bar', figsize=(20,10))
plt.ylabel('number of people with tag')
plt.title('Tag distribution for people with EN tag', fontsize=24)
plt.savefig('../reports/en_tag_distribution.png')

In [ ]:
%matplotlib inline

prs_value_counts_es[prs_value_counts_es.index[:100]].plot(kind='bar', figsize=(20,10))
plt.ylabel('number of people with tag')
plt.title('Tag distribution for people with ES tag', fontsize=24)
plt.savefig('../reports/es_tag_distribution.png')

In [ ]:
## make a column st for every user we have their deduped names, not sorted

prsf_en = prsf_en[prsf_en['column_meta_data_id'].isin([37337, 37341, 37336, 37324, 38561, 39943, 39645])]
prsf_es = prsf_es[prsf_es['column_meta_data_id'].isin([37337, 37341, 37336, 37324, 38561, 39943, 39645])]

prsf_en['combined_items'] = prsf_en['person_id']\
                           .apply(lambda x: "---".join(prsf_en[prsf_en['person_id'] == x]['item_name'].values ), meta=str).compute()

In [ ]:

prsf_es['combined_items'] = prsf_es['person_id']\
                           .apply(lambda x: "---".join(prsf_es[prsf_es['person_id'] == x]['item_name'].values.compute() ), meta=str).compute()

In [ ]:
help(person_ranking_summ_filtered.drop_duplicates())

In [ ]:
pd.__version__

In [ ]:
import dask 
dask.__version__

In [ ]:
person_ranking_summ

In [ ]:
prs_value_counts = person_ranking_summ.item_name.value_counts().compute()
for x in prs_value_counts.index:
    print(x, prs_value_counts[x])

In [ ]:
with open('Z:\\segment.tsv', 'r') as fp:
    for line in fp:
        print(line)

In [ ]:
matched_segment = dd.read_csv('Z:\\matched_segment.tsv', delimiter='\t', names=['event_id', 'segment_id', 'person_id', 'event_date'],
                             dtype={'touch1':np.int64, 'event_id':np.int64, 'touch2':np.int64})

In [ ]:
matched_segment

In [ ]:
len(matched_segment)


In [ ]:
matched_segment.segment_id.value_counts().compute()

In [ ]:
matched_segment.person_id.value_counts().compute()

In [ ]:
'''
touch id 1 - int64   pretty sure event_id?
segment id   - int64  
touch id 2 - int64   pretty sure person id?
session date - datetime pretty sure event date?
'''

In [ ]:
with open('Z:\\matched_segment.tsv', 'r', encoding='utf-8') as f:
    for i in range(2000000):
        l = f.readline()
        if i % 10000 == 0:
            print(l)

In [ ]:
fp_out = open('Z:\\person_ranking_summary_filtered.tsv', 'w', encoding='utf-8')
fp_in = open('Z:\\person_ranking_summary.tsv')
fp_in.readline()
ctr = 0
with open('Z:\\person_ids_gt_one_sorted.csv', 'r') as fp:
    for row in fp:
        in_row = fp_in.readline()
        #if ctr % 50000 == 0:
        #    print(ctr)
        #ctr+=1
        while in_row.startswith(str(row.strip(' \n'))):
            fp_out.write(in_row)
            in_row = fp_in.readline()
            
fp_out.close()
            

In [ ]:
person_identifier = dd.read_csv('Z:\\person_identifier.tsv', delimiter='\t', 
                                names=['person_id', 'customer_id', 'identifier',
                                       'identifier_type', '?', 'active', 'last_modified',
                                       'db_last_modified'])

In [ ]:
len(person_identifier)

In [ ]:
### person columns
person_headers = '''
id	engagement_score	last_touch	first_touch	customer_id	first_time_visitor	subscriber_status	subscription_plan	subscription_start_date	subscription_end_date	last_subscription_plan	articles_remaining	external_score	primary_identiifer	primary_identify_type	unknown	persona	active	last_modified_date	anonymous_visitor	do_not_track	identifiers_json	custom_field_1	custom_field_2	custom_field_3	custom_field_4	custom_field_5	custom_field_6	custom_field_7	custom_field_8	custom_field_9	custom_field_10	custom_field_11	custom_field_12	custom_field_13	custom_field_14	custom_field_15	custom_field_16	custom_field_17	custom_field_18	custom_field_19	custom_field_20	custom_field_21	custom_field_22	custom_field_23	custom_field_24	custom_field_25	custom_field_26	custom_field_27	custom_field_28	custom_field_29	custom_field_30	custom_field_31	custom_field_32	custom_field_33	custom_field_34	custom_field_35	custom_field_36	custom_field_37	custom_field_38	custom_field_39	custom_field_40	custom_field_41	custom_field_42	custom_field_43	custom_field_44	custom_field_45	custom_field_46	custom_field_47	custom_field_48	custom_field_49	custom_field_50	unknown2	unknown3	db_last_modifed_date'''.split('\t')

In [ ]:
person_headers

In [ ]:
person = dd.read_csv('Z:\\person.tsv', delimiter='\t', 
                                names=person_headers, 
                    dtype={'custom_field_12': 'string',
       'custom_field_13': 'string',
       'custom_field_4': 'string',
       'custom_field_5': 'string',
       'custom_field_6': 'string',
       'custom_field_7': 'string',
       'custom_field_10': 'string',
       'engagement_score': 'float64',
       'persona': 'string',
       'unknown2': 'string',
       'unknown3': 'string'})

In [ ]:
len(person)

In [ ]:
person['custom_field_35'].value_counts().compute()

In [ ]:
person['first_time_visitor'].value_counts().compute()#first_time_visitor

In [ ]:
person['anonymous_visitor'].value_counts().compute()

In [ ]:
person['unknown3'].value_counts().compute()

In [ ]:
person['custom_field_8'].value_counts().compute()

In [ ]:
person['custom_field_11'].value_counts().compute()

In [ ]:
person['unknown2'].value_counts().compute()

In [ ]:
person['persona'].value_counts().compute()

In [ ]:
person['custom_field_10'].value_counts().compute()

In [ ]:
person['custom_field_7'].value_counts().compute()

In [ ]:
person['custom_field_6'].value_counts().compute()

In [ ]:
person['custom_field_5'].value_counts().compute()

In [ ]:
person['custom_field_4'].value_counts().compute()

In [ ]:
person['custom_field_13'].value_counts().compute()

In [ ]:
person['custom_field_12'].value_counts().compute()

In [ ]:
import pickle as pkl

In [ ]:
with open('C:\\Users\dssgsolve\Solve\Joao\solve-ilao\data\processed\person_ids_appear_more_than_once.pkl', 'rb') as fp:
    person_ids_gt_one = pkl.load(fp)

In [ ]:
with open('Z:\\person_ids_gt_one_touch.csv', 'w') as fp:
    for x in person_ids_gt_one:
        fp.write('{}\n'.format(x))

In [ ]:
len(person_ids_gt_one)

In [ ]:
event_headers = 'event_id	customer_id	touch_ud	person_id	event_category_id	event_name	engagement_score	event_date	event_category_type	page_url	image_thumb_url	content_type	content_section	content_title	offer_id	campaign_id	keywords	content_pub_date	capture_identifier	empty	referrer	empty2	account_id	event_categort_type	content_id	customer_site_id	customer_site_external_id	last_modified_date	empty3	empty4	empty5	empty6	empty7	db_last_modified_date	timezone	custom_field1	custom_field_2	Language	Outgoing_link	content_section	download_link	get-legal-help	empty10	empty11	role	content_type	legal_position	empty15	empty16	empty17	empty18	empty19	empty20	empty21	empty22	empty23	empty24	empty25	empty26	empty27																																												db_last_modified_date	'.split('\t')

In [ ]:
event_headers

In [ ]:
with open('Z:\\event.tsv', 'r', encoding='utf-8') as fp:
    print(fp.readline())

In [ ]:
with open('Z:\\person.tsv', 'r', encoding='utf-8') as f:
    #for l in f:
    #    if '2019-03-06 18:20:08.580000' in l:
    #        print(l)
    #        print(l.count('\t'))
    for i in range(20):#range(200000):
        l = f.readline()
        print(l)
        #if i % 10000 == 0:
        #    print(l)
    '''
    for i in range(10000000):
        l = f.readline()
        n_tabs = l.count('\t')
        if n_tabs != 75:
            print('Line %i has %i tabs:' % (i, n_tabs))
            print(repr(l))
    '''

In [ ]:
with open('Z:\\person.tsv', 'r', encoding='utf-8') as f:
    l = f.readline()
    print(l)
    print(l.split('\t'))
    print(l.count('\t'))

In [ ]:
print('hi')

In [ ]:
# some columns look like floats at first because they are all NA, though they are strings (utm_...)
# and dma_code looks like an int at first but has NA
# for these we need to specify the type to read correctly
touch = dd.read_csv('Z:\\touch.tsv', delimiter='\t', encoding='utf-8',
                    dtype={'utm_medium':str, 'utm_terms':str, 'utm_content':str, 'utm_name':str, 'url_domain':str,
                          'dma_code':str,'customer_id': 'object',
                                           'tbd': 'object',
                                       'touch_duration.1': 'object'},
                    assume_missing=True)#,
                    #names=touch_columns)

In [ ]:
len(touch)

In [ ]:
with open('Z:\\touch.tsv', 'r') as f:
    for i in range(1000000):
        l = f.readline()
        if i % 10000 == 0:
            print(l)

even with encoding set to utf-8, we quickly see issues. For instance, a date falls on the field tbd. That date is 2019-03-04 01:54:30.000000. Could we find the lines around it and see if there's anything weird?

In [ ]:
with open('Z:\\touch.tsv', 'r', encoding='utf-8') as f:
    for i in range(200000):
        l = f.readline()
        if i % 10000 == 0:
            print(l)

In [ ]:
with open('Z:\\touch.tsv', 'r', encoding='utf-8') as f:
    for i in range(160000):
        l = f.readline()
        if i>150000 and i % 1000 == 0:
            print(l)

In [ ]:
with open('Z:\\touch.tsv', 'r', encoding='utf-8') as f:
    for i in range(10000000):
        l = f.readline()
        n_tabs = l.count('\t')
        if n_tabs != 58:
            print('Line %i has %i tabs:' % (i, n_tabs))
            print(repr(l))

So, I think ^ is _very_ interesting!

There are essentially two problems:

1) Some fields have \n or \t inside quotes. That should be easy to handle!

2) Some instances of the thirtieth field, whatever it is, have \n. That seems to me harder to completely fix, and we may end up with those lines broken in two...

Update after the fact: I just fixed lines by hand - mostly deleted the extra lines after some fields. Did this by printing as above and then jumping to the affected line on 010Editor.

In [ ]:
touch_sample = pd.read_csv('../data/raw/acquia_samples/updated_data/touch.tsv', delimiter='\t',
                           dtype={'utm_medium':str, 'utm_terms':str, 'utm_content':str, 'utm_name':str, 'url_domain':str,
                                  'dma_code':str},
                           parse_dates=['touch_date', 'db_last_modified_date', 'db_last_modified_date.1'],
                           names=touch_columns)

In [ ]:
counts_by_user = touch_sample.person_id.value_counts()

In [ ]:
counts_by_user

So, there are very few 1s, even in the sample data, which is surprising. But actually it's not: many touches get duplicated, somehow. See, for instance:

In [ ]:
touch_sample[touch_sample.person_id == 2066689106]

Three of these four are the same. So one task is to deduplicate these touches. To preserve all the data we should try to do this in a smart way, eg get the maximum duration, ignore NaN in favor of rows with that data, etc. For now I will not try to do this in a smart way -- let's try to keep just one of the rows, whichever it is.

TODO: make this better.

In [ ]:
np.abs(touch_sample.touch_date.iloc[2782]-touch_sample.touch_date.iloc[2769])

In [ ]:
touch_sample_person_ids = touch_sample.person_id.unique()

In [ ]:
to_delete = pd.Series(index=touch_sample.index, dtype=bool)

In [ ]:
fuzz = pd.Timedelta('00:00:01')
for id_p in touch_sample_person_ids:
    times = []
    sample_one_id = touch_sample[touch_sample.person_id == id_p]
    for idx, row in sample_one_id.iterrows():
        this_time = row.touch_date
        for t in times:
            if np.abs(t-this_time) < fuzz:
                to_delete.iloc[idx] = True
                break
        if not to_delete.iloc[idx]:
            times.append(this_time)

In [ ]:
np.sum(to_delete)

In [ ]:
touch_sample_deduped = touch_sample[~to_delete]

In [ ]:
counts_by_user = touch_sample_deduped.person_id.value_counts()

In [ ]:
counts_by_user_hist = counts_by_user.value_counts()

In [ ]:
touch.loc[10000].compute()

In [ ]:
touch.tail()

In [ ]:
touch_person_ids = touch.person_id.unique()
to_delete = pd.Series(index=touch.index, dtype=bool)

Some type issue which makes me thing columns go misaligned somewhere in the file? This is strange. Will investigate later. The code below won't run until that's solved:

In [ ]:
fuzz = pd.Timedelta('00:00:01')
for id_p in touch_person_ids:
    times = []
    sample_one_id = touch[touch.person_id == id_p]
    for idx, row in sample_one_id.iterrows():
        this_time = row.touch_date
        for t in times:
            if np.abs(t-this_time) < fuzz:
                to_delete.iloc[idx] = True
                break
        if not to_delete.iloc[idx]:
            times.append(this_time)